In [1]:
#importando bibliotecas necessárias no projeto
from sklearn import svm
from keras.models import Sequential
from keras.utils import plot_model
from keras import regularizers, optimizers
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.python.client import device_lib
from sklearn.svm import SVC
from keras import utils as np_utils
from keras import backend
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt 

Using TensorFlow backend.


### Carregando conjuntos de treino e teste

In [2]:
from re import search
import numpy as np
from subprocess import getoutput as gop
import glob
import pandas as pd
import csv

# identificando pastas
folders = {
    'large_train': 'dataset/large_train',
    'large_test': 'dataset/large_test',
}

def load_data(dataset):
    ch_names = []
    create_ch_name = False
    
    data_dir = gop('ls {}'.format(folders[dataset])).split('\n')
    # 1ª dimensão dos dados contendo os sujeitos
    subjects = list()
    subjects_alc = list()
    subjects_ctrl = list()
    
    for types in data_dir:
        files = gop('ls {}/{}'.format(folders[dataset], types)).split('\n')
        # 2ª dimensão dos dados contendo as sessões (trials)
        trials = list()
        is_alc = True
        
        for f in files:
            arquivo = open('{}/{}/{}'.format(folders[dataset], types, f))
            text = arquivo.readlines()
            
            alc = search('co2a', text[0])
            ctrl = search('co2c', text[0])
            
            if ctrl:
                is_alc = False
            # 3ª dimensão dos dados contendo os canais (eletrodos)
            chs = list()
            # 4ª dimensão dos dados contendo os valores em milivolts
            values = list()
            for line in text:
                t = search('(?P<ch_name>\w{1,3}) chan \d{1,2}', line)
                p = search('^\d{1,3}\ \w{1,3}\ \d{1,3}\ (?P<value>.+$)', line)
                                    
                if p:
                    values.append(float(p.group('value')))
                # mudou para outro eletrodo
                elif t:
                    if values:
                        chs.append(values)
                        values = list()
                    if not create_ch_name:
                        ch_names.append(t.group('ch_name').lower())
            
            create_ch_name = True
            chs.append(values)
            trials.append(chs)
            arquivo.close()
            
        if is_alc:
            subjects_alc.append(trials)
            md_alc = np.average(trials, axis=0)
        else:
            subjects_ctrl.append(trials)
            md_ctrl = np.average(trials, axis=0)
            
    data_alc = np.array(subjects_alc)
    data_ctrl = np.array(subjects_ctrl)
    
#     data.tofile('./dataset_csv/small_data.csv', sep=',', newline='')
        
    return data_alc, md_alc, data_ctrl, md_ctrl, ch_names

In [3]:
lg_train_alc, md_alc_train, lg_train_ctrl, md_ctrl_train, ch_names = load_data('large_train')
lg_test_alc, md_alc_test, lg_test_ctrl, md_ctrl_test, ch_names = load_data('large_test')

### Modelo de rede neural

In [7]:
model = Sequential()
num_input = lg_train_alc.shape[1]

model.add(Dense(units=100, activation='relu', input_dim=num_input))
model.add(Dense(units=59, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = model.fit(md_alc_train, md_ctrl_train, validation_split=0.30, epochs=50, batch_size=50, verbose=1)

loss_and_metrics = model.evaluate(md_alc_test, md_ctrl_test, batch_size=16)
print("\n Taxa de acerto: %.2f%%" % (loss_and_metrics[1]*100))

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

print("\n Função de Custo: %.2f%%" % (loss_and_metrics[0]))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

ValueError: Error when checking input: expected dense_7_input to have shape (30,) but got array with shape (256,)